In [2]:
import keras
keras.__version__
import tensorflow as tf
tf.__version__

In [3]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

# specifically for cnn
from tensorflow.keras.layers import Dropout, Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import InputLayer
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

from tensorflow.keras.applications import VGG16

# import tf.keras as keras

In [4]:
os.listdir('../input/fire-dataset/fire_dataset')

In [5]:
def make_train_data(label,DIR):
    for img in tqdm(os.listdir(DIR)):
        path = os.path.join(DIR,img)
        img = Image.open(path)
        img = img.resize((IMG_SIZE,IMG_SIZE))
        img =img.convert('RGB')
        img= np.array(img)
        X.append(img)
        Z.append(str(label))

In [6]:


X=[]
Z=[]
IMG_SIZE=256
FIRE='../input/fire-dataset/fire_dataset/fire_images'
NOTFIRE='../input/fire-dataset/fire_dataset/non_fire_images'

make_train_data('1',FIRE)
make_train_data('0',NOTFIRE)



In [7]:
X=np.array(X)
fig,ax=plt.subplots(2,5)
plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0)
fig.set_size_inches(10,10)

for i in range(2):
    for j in range (5):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title(Z[l])
        ax[i,j].set_aspect('equal')


In [8]:
le=LabelEncoder()
Y=le.fit_transform(Z)
# print(Y)
Y=to_categorical(Y,2)
print(Y[800])
# non fire-0
# fire-1
print('RGB value of image',X[0][0][0])

In [9]:
#example preprocessing


path="../input/fire-dataset/fire_dataset/fire_images/fire.1.png"
import math
x=Image.open(path)
x=x.resize((IMG_SIZE,IMG_SIZE))
x =x.convert('RGB')
plt.imshow(x)
arr=np.array(x)
print(arr.shape)
#segmentaion HSI

for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        r=arr[i,j,0]
        g=arr[i,j,1]
        b=arr[i,j,2]
        if(r>g and g>b ):
            pass
        else:
            arr[i,j]=[0,0,0]
        
# res=np.zeros((IMG_SIZE,IMG_SIZE,3))
# for i in range(IMG_SIZE):
#     for j in range(IMG_SIZE):
#         s=np.sum(arr[i,j])
#         r=arr[i,j,0]/s
#         g=arr[i,j,1]/s
#         b=arr[i,j,2]/s
#         res[i,j,2]=(s*1.0)/3
#         res[i,j,1]=(1-(3.0 * min([r,g,b])))*100
#         d=math.sqrt((r-g)*(r-g)+(r-b)*(g-b))
#         val= (0.5*(r-g+r-b))/(d+0.0000001)
#         theta = math.degrees(math.acos(val))
#         if(b<=g):
#             res[i,j,0]=(theta)
#         else:
#             res[i,j,0]=(360-theta)
#         d=math.sqrt((r-g)*(r-g)+(r-b)*(g-b))
#         val= (0.5*(r-g+r-b))/(d+0.0000001)
#         theta = math.degrees(math.acos(val))
#         if(b<=g):
#             res[i,j,0]=(theta)
#         else:
#             res[i,j,0]=(360-theta)
       

In [10]:
!ls '../input/fire-dataset/fire_dataset/fire_images/fire.1.png'

In [11]:
plt.imshow(arr)

In [12]:
def segmentation(arr):
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            r=arr[i,j,0]
            g=arr[i,j,1]
            b=arr[i,j,2]
            if(r>g and g>b and r>200):
                pass
            else:
                arr[i,j]=[0,0,0]
    return arr

In [13]:
X_new=[]
for mat in X:
    X_new.append(segmentation(mat))


In [14]:
X_new=np.array(X_new)
fig,ax=plt.subplots(2,5)
plt.subplots_adjust(bottom=0.3, top=0.7, hspace=0)
fig.set_size_inches(10,10)

for i in range(2):
    for j in range (5):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X_new[l])
        ax[i,j].set_title(Z[l])
        ax[i,j].set_aspect('equal')



In [15]:
x_train,x_test,y_train,y_test=train_test_split(X_new,Y,test_size=0.25,random_state=1337)

np.random.seed(42)
rn.seed(42)
#tf.set_random_seed(42)

In [16]:
base_model=VGG16(include_top=False, weights='imagenet',input_shape=(IMG_SIZE,IMG_SIZE,3), pooling='max')
base_model.summary()

In [17]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.20))
model.add(Dense(2048,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [18]:

epochs=32
batch_size=128
base_model.trainable=True 
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
model.compile(optimizer=Adam(lr=1e-5),loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [19]:
History = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test,y_test),callbacks=[es])

In [20]:
model.save('../working/model2.h5')

In [21]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()
modelName = "fireClassification"
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(str(modelName+".tflite"), "wb").write(tflite_model)

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
path="../input/fire-dataset/fire_dataset/fire_images/fire.2.png"
x=Image.open(path)
x=x.resize((IMG_SIZE,IMG_SIZE))
x=x.convert('RGB')
plt.imshow(x)
x = img_to_array(x)
x= segmentation(x)
# plt.imshow(x)
x = np.expand_dims(x, axis=0)
array = model.predict(x)
print(array)
result = array[0]
answer = np.argmax(result)
print(answer)

In [ ]:
count=0
for x in X_new:
#     plt.imshow(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
#     print(array)
    result = array[0]
    answer = np.argmax(result)
    if answer :
        count+=1
#     print(answer)

In [ ]:
print(count,X_new.shape[0]-count)